## This notebook is a demo for

1. Generate a Graph (Line Graph / Random Graph).

2. Calculate the covariance matrix for the graph with a graph kernel function (with known hyperparameters).

3. Sample data points from a GP on this covariance matrix.

4. Fit a GP with the exact kernel expression and learn the hyperparameters.

In [8]:
import tensorflow as tf
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import networkx as nx
from gpflow.utilities import print_summary
import tensorflow_probability as tfp
import seaborn as sns

In [9]:
import sys
import os
project_root = os.path.abspath("..")
sys.path.append(project_root)

In [10]:
from efficient_graph_gp.graph_kernels import diffusion_kernel, get_normalized_laplacian, generate_noisy_samples
from efficient_graph_gp.gpflow_kernels import GraphDiffusionKernel
from utils import plot_network_graph, plot_gp_fit

In [11]:
# Parameters
num_nodes = 30
graph_type = 'random' # 'line', 'random'

In [12]:
if graph_type == 'line':
    adjacency_matrix = np.eye(num_nodes, k=1) + np.eye(num_nodes, k=-1)  # Circular adjacency matrix
elif graph_type == 'random':
    probability = 0.1  # Probability of edge creation
    G = nx.erdos_renyi_graph(num_nodes, probability, directed=False)  # Ensure the graph is undirected
    adjacency_matrix = nx.to_numpy_array(G)  # Convert to adjacency matrix

In [13]:
def gp_inference(X,Y,X_new, graph_kernel):
    model = gpflow.models.GPR(data=(X, Y), kernel=graph_kernel, mean_function=None)
    model.likelihood.variance.prior = tfp.distributions.LogNormal(loc=np.log(0.07), scale=0.5)
    gpflow.optimizers.Scipy().minimize(model.training_loss, model.trainable_variables)
    mean, variance = model.predict_f(X_new)
    stddev = tf.sqrt(variance)
    return model, mean, stddev

def plot_kernel_heatmap(kernel_matrix, title, ax):
    sns.heatmap(kernel_matrix, ax=ax, cmap='viridis')
    ax.set_title(title)
    ax.set_xlabel('Node Index')
    ax.set_ylabel('Node Index')

In [14]:
def demo(beta_sample, noise_std=0.1):
    clear_output(wait=True)  # Clear previous output
    # Generate noisy samples
    K_true = diffusion_kernel(adjacency_matrix, beta_sample)
    Y_noisy = generate_noisy_samples(K_true,noise_std=noise_std)
    X = tf.convert_to_tensor(np.arange(num_nodes, dtype=np.float64).reshape(-1, 1))  # Input features (nodes)
    X_new = tf.convert_to_tensor(np.arange(num_nodes).reshape(-1, 1), dtype=tf.float64)  # New input features for prediction
    Y = tf.convert_to_tensor(Y_noisy, dtype=tf.float64)  # Noisy sampled data
    graph_kernel = GraphDiffusionKernel(adjacency_matrix)
    model, mean, stddev = gp_inference(X, Y, X_new, graph_kernel)
    print_summary(model)
    
    learned_beta = model.kernel.beta.numpy()
    K_fitted = diffusion_kernel(adjacency_matrix, learned_beta)
    fig, ax = plt.subplots(2, 2, figsize=(16, 12))
    plot_gp_fit(X, Y, X_new, mean, stddev, beta_sample, ax[0,0])
    plot_network_graph(adjacency_matrix, ax[0,1])
    plot_kernel_heatmap(K_true, 'Ground Truth Kernel', ax[1,0])
    plot_kernel_heatmap(K_fitted, 'Fitted Kernel', ax[1,1])

beta_slider = widgets.FloatSlider(value=3.0, min=0.1, max=10.0, step=0.1, description='Beta:')
noise_std_slider = widgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='Noise std:')
ui = widgets.VBox([beta_slider, noise_std_slider])
out = widgets.interactive_output(demo, {'beta_sample': beta_slider, 'noise_std': noise_std_slider})
display(ui, out)


Output()